# Import Packages

In [33]:
import pandas as pd

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset

import torchinfo
from torch_snippets import Report
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.has_mps:
    device = "mps"

# Read Dataset

In [31]:
general = pd.read_csv("../datasets/general.csv")
room = pd.read_csv("../datasets/room.csv")
categorical = pd.read_csv("../datasets/categorical.csv")
labels = pd.read_csv("../datasets/label.csv")

In [ ]:
general

In [ ]:
room

In [ ]:
categorical

In [ ]:
labels

# Create a class for preparing data

In [ ]:
class PropertyDataset(Dataset):
    def __init__(self, general, room, categorical, labels):
        self.general = general
        self.room = room
        self.categorical = categorical
        self.labels = labels

        self.complete_encoder = LabelEncoder()
        self.complete_encoder.fit(self.labels.Completed)

    def __getitem__(self, item):
